# Семинар 4: нейронная сеть

*Задание основано на ноутбуке курса Школы Анализа Данных по глубинному обучению.*

На этом семинаре мы напишем нейронную сеть с нуля. Реализуемая нами архитектура в целом схожа с тем, как выглядит нейронная сеть в Pytorch.

Этот семинар состоит из двух ноутбуков: Modules.ipynb, в котором будет находиться реализация нейронной сети, и этот ноутбук, в котором вы будете выполнять все эксперименты с нейронной сетью. Пожалуйста, не меняйте прототипы функций.

## Введение

Нейронные сети стали популярными по многим причинам, но одна из них - это модульность. Нейронные сети состоят из модулей (слоев), каждый слой реализует какую-то функциональность. Комбинируя имеющиеся слои можно реализовать state-of-art архитектуру с помощью уже имеющейся библиотеки (Pytorch, Tensorflow итд). Часто для реализации множества прорывных современных идей достаточно определить новый слой, или даже просто слегка изменить уже имеющийсяю

Давайте для начала посмотрим на нейронную сеть как на черный ящик (нас не интересует как он устроен, но когда мы просим его что-то сделать - он вежливо выполняет просьбу). Какую функциональность должна иметь нейронная сеть? Такую же как и остальные модели машинного обучения, а именно:

1) По данному входу (input) она должна выдавать предсказания (output)

2) Она должна быть обучаемой (уметь подстраиваться под имеющиеся данные)

Остановимся пока на первом пункте. Назовем метод, который по данному входу дает какие-то предсказания **forward** (если вы делали домашнее задание - то это в точности метод **forward_pass** из него).



NN.forward(input)

## Forward

In [ ]:
%matplotlib inline
from time import time, sleep
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

## Dumb forward pass



Реализуйте метод dumb_forward в `Modules.ipynb` для модуля Linear.

In [ ]:
%run modules.ipynb
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

Протестируйте ваш dumb_forward на искуственно сгенерированных данных.

In [ ]:

input_size = 4
num_classes = 3
num_inputs = 5

def init_linear():
    np.random.seed(0)
    return Linear(input_size, num_classes)

def init_toy_data():
    np.random.seed(1)
    X = 10 * np.random.randn(num_inputs, input_size)
    y = np.array([0, 1, 2, 2, 1])
    return X, y

linear = init_linear()
X, y = init_toy_data()

In [ ]:
output = linear.dumb_forward(X)
print('Your output:')
print(output)
print()
print('correct output:')
correct_output = np.asarray(
[[ -1.54788446,  -6.00658097,   6.39369587],
 [ -3.07885716,  -8.08969546,  11.56531411],
 [  0.19697038,  -9.59100515,   5.43998673],
 [ -0.31239372,  -5.33085678,  1.94239605],
 [ -1.66825993,   1.10786278,   0.51528511]])
print(correct_output)
print()

# The difference should be very small. We get < 1e-7
print('Difference between your scores and correct scores:')
print(rel_error(output, correct_output))

Проверим, что результат совпадает с тем, что мы получили ранее.

In [ ]:
sequential = Sequential()
linear_small = Linear(n_in=2, n_out=2)
linear_small.W = np.array([[0.9, 0.3], [0.2, 0.8]])
linear_small.b = np.zeros(2)
sequential.add(linear_small)
sequential.add(Sigmoid())

In [ ]:
X_small = np.array([[1, 0.5]])
output_small = sequential.forward(X_small)
print('Your output:')
print(output_small)
print()
print('correct output:')
correct_output_small = np.asarray([[0.7408, 0.6457]])
print(correct_output_small)
print()

print('Difference between your scores and correct scores:')
print(rel_error(output_small, correct_output_small))

# Forward

Реализуйте метод forward в модуле Linear с помощью матричных операций библиотеки numpy.

In [ ]:
%run modules.ipynb

output = linear.forward(X)
print('Your output:')
print(output)
print()
print('correct output:')
print(correct_output)
print()

# The difference should be very small. We get < 1e-7
print('Difference between your scores and correct scores:')
print(rel_error(output, correct_output))

# Backward

Реализуйте методы updateGradInput и accGradParameters для модуля Linear.

In [ ]:
%run modules.ipynb
from gradient_check import eval_numerical_gradient

# Используйте численное дифференцирования чтобы проверить вашу реализацию подсчета градиента
# Если ваша реализация верна, то относительная ошибка будет не больше 1e-5

linear = init_linear()
output = linear.forward(X)
grad_output = np.ones_like(output)

def calc_numerical_grad_for_linear(X=X, linear=linear, eps=1e-5):
    n_objects, input_size = X.shape
    n_classes, _ = linear.W.shape
    W = linear.W.copy()
    W_grad = np.zeros_like(W)
    for row_idx in range(num_classes):
        for column_idx in range(input_size):
            linear.W = W.copy()
            linear.W[row_idx][column_idx] += eps
            right_output = linear.forward(X).sum()
            linear.W = W.copy()
            linear.W[row_idx][column_idx] -= eps
            left_output = linear.forward(X).sum()
            W_grad[row_idx][column_idx] = (right_output - left_output) / (2 * eps)
    X_grad = np.zeros_like(X)
    for obj_idx in range(n_objects):
        for column_idx in range(input_size):
            right_X = X.copy()
            right_X[obj_idx][column_idx] += eps
            right_output = linear.forward(right_X).sum()
            left_X = X.copy()
            left_X[obj_idx][column_idx] -= eps
            left_output = linear.forward(left_X).sum()
            X_grad[obj_idx][column_idx] = (right_output - left_output) / (2 * eps)
    return W_grad, X_grad

W_grad_numerical, X_grad_numerical = calc_numerical_grad_for_linear()
X_grad = linear.backward(X, np.ones_like(output))
W_grad = linear.getGradParameters()[0]

print('Your gradient w.r.t W:')
print(W_grad)
print()
print('correct gradient w.r.t W:')
print(W_grad_numerical)
print()

print('Difference between your scores and correct scores:')
print(rel_error(X_grad, X_grad_numerical))
print('Your gradient w.r.t input:')
print(X_grad)
print()
print('correct gradient w.r.t input:')
print(X_grad_numerical)
print()

print('Difference between your scores and correct scores:')
print(rel_error(X_grad, X_grad_numerical))